In [1]:
## importing the base file

import csv

file_path = r"C:\Users\adity\Documents\Python projects\COVID-19 effect on Liver Cancer Prediction\data1.csv"

with open(file_path, 'r') as csv_file:
    reader = csv.reader(csv_file)

In [2]:
import datetime
import os
import collections

import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from matplotlib import pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import (
    accuracy_score,
    auc,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (cross_validate, train_test_split)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


In [3]:
try:
    df = pd.read_csv(file_path, encoding = 'utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding = 'ISO-8859-1')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding = 'cp1252')

print(df.shape)


(450, 27)


In [4]:
pd.set_option('display.max_columns', None)
df

,Cancer,Year,Month,Bleed,Mode_Presentation,Age,Gender,Etiology,Cirrhosis,Size,HCC_TNM_Stage,HCC_BCLC_Stage,ICC_TNM_Stage,Treatment_grps,Survival_fromMDM,Alive_Dead,Type_of_incidental_finding,Surveillance_programme,Surveillance_effectiveness,Mode_of_surveillance_detection,Time_diagnosis_1st_Tx,Date_incident_surveillance_scan,PS,Time_MDM_1st_treatment,Time_decisiontotreat_1st_treatment,Prev_known_cirrhosis,Months_from_last_surveillance
0,Y,Prepandemic,1,N,Surveillance,68,M,NAFLD,Y,22.0,II,A,NaN,Ablation,32.73,Alive,NaN,Y,Consistent,US,0.47,NaN,0.0,0.70,NaN,Y,7.333333
1,Y,Prepandemic,1,N,Surveillance,70,M,ARLD,Y,40.0,I,D,NaN,Supportive care,3.03,Dead,NaN,Y,Consistent,US,NaN,NaN,2.0,NaN,NaN,Y,4.033333
2,Y,Prepandemic,1,N,Surveillance,64,M,ARLD,Y,52.0,IV,B,NaN,Medical,14.97,Dead,NaN,Y,Consistent,US,NaN,NaN,0.0,NaN,NaN,Y,5.900000
3,Y,Prepandemic,1,N,Incidental,73,M,ARLD,Y,80.0,IV,C,NaN,Supportive care,1.40,Dead,Secondary careacute,N,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Y,NaN
4,Y,Prepandemic,1,N,Incidental,66,F,ARLD,Y,60.0,I,0,NaN,Supportive care,32.50,Alive,Secondary careacute,N,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Y,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,N,Pandemic,12,NaN,Symptomatic,72,F,NaN,NaN,NaN,NaN,NaN,III,Supportive care,1.20,Dead,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Y,NaN
446,N,Pandemic,12,NaN,Symptomatic,63,M,NaN,NaN,75.0,NaN,NaN,IV,Medical,9.67,Alive,NaN,NaN,NaN,NaN,2.93,NaN,0.0,1.63,1.63,Y,NaN
447,N,Pandemic,12,NaN,Symptomatic,79,F,NaN,NaN,63.0,NaN,NaN,IV,Supportive care,1.47,Dead,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Y,NaN
448,N,Pandemic,12,NaN,Symptomatic,84,F,NaN,NaN,65.0,NaN,NaN,IV,Medical,1.67,Dead,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Y,NaN


In [5]:
df.columns

Index(['Cancer', 'Year', 'Month', 'Bleed', 'Mode_Presentation', 'Age',
       'Gender', 'Etiology', 'Cirrhosis', 'Size', 'HCC_TNM_Stage',
       'HCC_BCLC_Stage', 'ICC_TNM_Stage', 'Treatment_grps', 'Survival_fromMDM',
       'Alive_Dead', 'Type_of_incidental_finding', 'Surveillance_programme',
       'Surveillance_effectiveness', 'Mode_of_surveillance_detection',
       'Time_diagnosis_1st_Tx', 'Date_incident_surveillance_scan', 'PS',
       'Time_MDM_1st_treatment', 'Time_decisiontotreat_1st_treatment',
       'Prev_known_cirrhosis', 'Months_from_last_surveillance'],
      dtype='object')

In [6]:
df.dtypes

Cancer                                 object
Year                                   object
Month                                   int64
Bleed                                  object
Mode_Presentation                      object
Age                                     int64
Gender                                 object
Etiology                               object
Cirrhosis                              object
Size                                  float64
HCC_TNM_Stage                          object
HCC_BCLC_Stage                         object
ICC_TNM_Stage                          object
Treatment_grps                         object
Survival_fromMDM                      float64
Alive_Dead                             object
Type_of_incidental_finding             object
Surveillance_programme                 object
Surveillance_effectiveness             object
Mode_of_surveillance_detection         object
Time_diagnosis_1st_Tx                 float64
Date_incident_surveillance_scan   

In [7]:
## preprocessing certain columns

df['Cancer'] = df['Cancer'].map({'Y' : 1, 'N': 0})

df.rename(columns = {'Year': 'Timeline'}, inplace = True)

df['Bleed'] = df['Bleed'].map({'Y' : 1, 'N': 0})
df['Bleed'] = df['Bleed'].fillna(1)

df['Mode_Presentation'] = df['Mode_Presentation'].map({'Surveillance' : 1, 'Incidental' : 2, 'Symptomatic' : 3})

df['Gender'] = df['Gender'].map({'M' : 0, 'F' : 1})

#df = df.dropna(subset = ['Etiology'])
df.drop(columns = ['Etiology'], inplace = True)

df['Cirrhosis'] = df['Cirrhosis'].map({'Y' : 1, 'N' : 0})
df['Cirrhosis'] = df['Cirrhosis'].fillna(0)

df['Size'] = df['Size'].fillna(0)

#df['HCC_TNM_Stage'] = df['HCC_TNM_Stage'].map({'I' : 1, 'II' : 2, 'IIIA+IIIB' : 3, 'IV' : 4})
#df['HCC_TNM_Stage'] = df['HCC_TNM_Stage'].fillna(5)
df.drop(columns = ['HCC_TNM_Stage'], inplace = True)

df['HCC_BCLC_Stage'] = df['HCC_BCLC_Stage'].map({'0' : 0, 'A' : 1, 'B' : 2, 'C' : 3, 'D' : 4})
df['HCC_BCLC_Stage'] = df['HCC_BCLC_Stage'].fillna(5)

df.drop(columns = ['ICC_TNM_Stage'], inplace = True)

df['Alive_Dead'] = df['Alive_Dead'].map({'Alive' : 1, 'Dead' : 0})

#df = df.dropna(subset = ['Type_of_incidental_finding'])
df.drop(columns = ['Type_of_incidental_finding'], inplace = True)

df['Surveillance_programme'] = df['Surveillance_programme'].map({'N' : 0, 'Y' : 1})
df['Surveillance_programme'] = df['Surveillance_programme'].fillna(2)

#df = df.dropna(subset = ['Surveillance_effectiveness'])
df.drop(columns = ['Surveillance_effectiveness'], inplace = True)

df.drop(columns = ['Mode_of_surveillance_detection'], inplace = True)

df.drop(columns = ['Time_diagnosis_1st_Tx'], inplace = True)

df.drop(columns = ['Date_incident_surveillance_scan'], inplace = True)

df.drop(columns = ['Time_MDM_1st_treatment'], inplace = True)

df.drop(columns = ['Time_decisiontotreat_1st_treatment'], inplace = True)

df['Months_from_last_surveillance'] = df['Months_from_last_surveillance'].fillna(0)
#df = df.dropna(subset = ['Months_from_last_surveillance'])


In [8]:
## this code chunk is used to deterine each columns' decision to preprocess or not

unique_values = df['Treatment_grps'].unique()
print(unique_values)

value_counts = df['Treatment_grps'].value_counts()
print(value_counts)

['Ablation' 'Supportive care' 'Medical' 'TACE' 'SIRT' 'OLTx' 'Resection'
 nan]
Treatment_grps
Supportive care    236
TACE                62
Medical             51
Ablation            40
Resection           30
SIRT                22
OLTx                 7
Name: count, dtype: int64


In [9]:
## final dataframe to be used for prediction analysis

df

,Cancer,Timeline,Month,Bleed,Mode_Presentation,Age,Gender,Cirrhosis,Size,HCC_BCLC_Stage,Treatment_grps,Survival_fromMDM,Alive_Dead,Surveillance_programme,PS,Prev_known_cirrhosis,Months_from_last_surveillance
0,1,Prepandemic,1,0.0,1,68,0,1.0,22.0,1.0,Ablation,32.73,1,1.0,0.0,Y,7.333333
1,1,Prepandemic,1,0.0,1,70,0,1.0,40.0,4.0,Supportive care,3.03,0,1.0,2.0,Y,4.033333
2,1,Prepandemic,1,0.0,1,64,0,1.0,52.0,2.0,Medical,14.97,0,1.0,0.0,Y,5.900000
3,1,Prepandemic,1,0.0,2,73,0,1.0,80.0,3.0,Supportive care,1.40,0,0.0,2.0,Y,0.000000
4,1,Prepandemic,1,0.0,2,66,1,1.0,60.0,0.0,Supportive care,32.50,1,0.0,0.0,Y,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,0,Pandemic,12,1.0,3,72,1,0.0,0.0,5.0,Supportive care,1.20,0,2.0,1.0,Y,0.000000
446,0,Pandemic,12,1.0,3,63,0,0.0,75.0,5.0,Medical,9.67,1,2.0,0.0,Y,0.000000
447,0,Pandemic,12,1.0,3,79,1,0.0,63.0,5.0,Supportive care,1.47,0,2.0,1.0,Y,0.000000
448,0,Pandemic,12,1.0,3,84,1,0.0,65.0,5.0,Medical,1.67,0,2.0,2.0,Y,0.000000


In [10]:
df.to_csv(r"C:\Users\adity\Documents\Python projects\COVID-19 effect on Liver Cancer Prediction\data2.csv", index = False)
